In [9]:
# !pip install nltk
!pip install wordcloud

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [20 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.9
  creating build\lib.win-amd64-3.9\wordcloud
  copying wordcloud\color_from_image.py -> build\lib.win-amd64-3.9\wordcloud
  copying wordcloud\tokenization.py -> build\lib.win-amd64-3.9\wordcloud
  copying wordcloud\wordcloud.py -> build\lib.win-amd64-3.9\wordcloud
  copying wordcloud\wordcloud_cli.py -> build\lib.win-amd64-3.9\wordcloud
  copying wordcloud\_version.py -> build\lib.win-amd64-3.9\wordcloud
  copying wordcloud\__init__.py -> build\lib.win-amd64-3.9\wordcloud
  copying wordcloud\__main__.py -> build\lib.win-amd64-3.9\wordcloud
  copying wordcloud\stopwords -> build\lib.win-amd64-3.9\wordcloud
  copying wordcloud\DroidSansMono.ttf -> build\lib.win-amd64-3.9\wordcloud
  UPDATING build\lib.win-amd64-3.9\wordcloud/_version.py
  set bu


  Using cached wordcloud-1.8.1.tar.gz (220 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for wordcloud
Failed to build wordcloud
  Running setup.py install for wordcloud: started
  Running setup.py install for wordcloud: finished with status 'error'


In [10]:
%matplotlib inline 

import re
import string
import nltk
import pickle
import warnings

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from wordcloud import WordCloud
from wordcloud import STOPWORDS

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

from sklearn.metrics import classification_report
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

from xgboost import XGBClassifier

ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
sns.set_style("white")
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', None)
warnings.simplefilter(action="ignore", category=FutureWarning)

In [12]:
df = pd.read_csv('E:\Study\Sem_6\Deep Learning\dl_ass02_v1\cyberbullying_tweets.csv')
df = df.rename(columns={"tweet_text":"text", "cyberbullying_type": "label"})

In [13]:
df

,text,label
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying
...,...,...
47687,"Black ppl aren't expected to do anything, depe...",ethnicity
47688,Turner did not withhold his disappointment. Tu...,ethnicity
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity


In [14]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47692 entries, 0 to 47691
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    47692 non-null  object
 1   label   47692 non-null  object
dtypes: object(2)
memory usage: 745.3+ KB


In [15]:
df.isnull().sum()


text     0
label    0
dtype: int64

In [16]:
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

STOPWORDS.update(['did', 'rt', 'will', 'im', 'thing'])

def show_wordcloud(data):
    words = ''
     
    for sentence in data:
     
        tokens = str(sentence).split()
         
        for i in range(len(tokens)):
            tokens[i] = tokens[i].lower()
         
        words += " ".join(tokens) + " "
     
    wordcloud = WordCloud(width = 800, height = 800, background_color ='white', min_font_size = 12).generate(words)
     
    plt.figure(figsize = (8, 8), facecolor = None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad = 0)
     
    plt.show()
    
def show_top_ngram(df_column):
    
    vectorizer = TfidfVectorizer(ngram_range=(2,2))

    ngrams = vectorizer.fit_transform(df_column)
    count_values = ngrams.toarray().sum(axis=0)
    vocab = vectorizer.vocabulary_
    df_ngram = pd.DataFrame(sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True)).rename(columns={0: 'frequency', 1:'bigram/trigram'})

    return df_ngram

def delete_url(text):
    links = re.findall(re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL), text)
    for link in links:
        text = text.replace(link[0], ' ')    
    return text

def delete_mention_tag(text):
               
    # filter kata yang mengandung penanda mention dan hashtag
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in ['@','#']:
                words.append(word)
    return ' '.join(words)

def preprocessing(text):
    text = text.lower()                    # convert ke lowercase
    text = delete_url(text)                # hapus URL/link
    text = delete_mention_tag(text)        # hapus entitas mention dan hashtags
    text = text.strip()
    text = " ".join([word for word in text.split() if not word in set(STOPWORDS)]) 
    text = re.sub(r" \d+ ", " ", text)     # hapus digit
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = re.sub(r"[^a-z ]", "", text)
    text = re.sub(r"  ", " ", text)
    return text

ModuleNotFoundError: No module named 'wordcloud'

In [8]:
df['text'] = df.apply(lambda row: preprocessing(row.text), axis=1)


NameError: name 'df' is not defined

In [ ]:
df['label'].value_counts().plot.bar(figsize=(6, 4));

plt.title('Distribution Target');
plt.xlabel('Label');
plt.ylabel('Count');

In [ ]:
df.loc[df['text'].str.contains("bully|bullied|bullies"), 'label'] = "not_cyberbullying"


In [ ]:
df['label'].value_counts().plot.bar(figsize=(6, 4));

plt.title('Distribution Target');
plt.xlabel('Label');
plt.ylabel('Count');

In [ ]:
df.loc[df['label'] == 'age', 'label'] = 'other_cyberbullying'

In [ ]:
df['label'].value_counts().plot.bar(figsize=(6, 4));

plt.title('Distribution Target');
plt.xlabel('Label');
plt.ylabel('Count');

In [ ]:
df['label'].value_counts().plot.bar(figsize=(10, 5));

plt.title('Distribution Target');
plt.xlabel('Label');
plt.ylabel('Count');

In [ ]:
df['text_length'] = df.apply(lambda row: len(row.text.split()), axis=1)


In [ ]:
df


In [ ]:
disp = df.boxplot(column = ['text_length'], grid = False, figsize=(10, 10))
plt.ylim(0, 80)
plt.show();

In [ ]:
df.plot.hist(bins=100, figsize=(10, 5))
plt.show();


In [ ]:
disp = df.boxplot(by ='label', column=['text_length'], grid = False, figsize=(10, 15))
plt.ylim(0, 70)
plt.show();

In [ ]:
show_wordcloud(df['text'])

In [ ]:
show_wordcloud(df[df['label'] == 'ethnicity']['text'])

In [ ]:
show_top_ngram(df[df['label'] == 'ethnicity']['text'])[:20]

In [ ]:
show_wordcloud(df[df['label'] == 'gender']['text'])

In [ ]:
show_top_ngram(df[df['label'] == 'gender']['text'])[:20]


In [ ]:
show_wordcloud(df[df['label'] == 'religion']['text'])

In [ ]:
show_top_ngram(df[df['label'] == 'religion']['text'])[:20]


In [ ]:
show_wordcloud(df[df['label'] == 'other_cyberbullying']['text'])


In [ ]:
show_top_ngram(df[df['label'] == 'other_cyberbullying']['text'])[:20]


In [ ]:
show_wordcloud(df[df['label'] == 'not_cyberbullying']['text'])


In [ ]:
show_top_ngram(df.loc[df['label'] == 'not_cyberbullying'][:5000]['text'])[:20]


In [ ]:
df = df.drop('text_length', axis=1)
